<a href="https://colab.research.google.com/github/madhusudanhasbe/Natural_Language_Processing_Lab/blob/main/22070126061_NLP_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assignment 2

Name: Madhusudan Hasbe\
PRN: 22070126061\
TY AIML A3

In [ ]:
import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'cleaned-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5699007%2F9391337%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240915%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240915T133852Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D38770253e0c3d73107639c8602bb62c4a45c4c192f95a66f66646e931319ead66414b9343a1b78780eb6e24cbafba4585cd87fe075e48a4a2a5db22fdd872e9e77e503108661ca52ed5f97557671a878afed36e88a30838ec8e85ca81e70675e5f23e8d33e9a5987b94721909b5c7ba4d0b096bfd2dc6d09f3e995c806a080d4903c8f2c1f0c201447624ee975f69ccfdafea71fe106789c2d15047d2b6a05516c1ab3d175f136f11a23ab93c0129fff8e55b1cf02465ec4b432ed09491f3b264f67b8e0b80fa93c4256ff1d12584815e09b9f988c89707ebb9f759632e8dd145f9efda69ee507ef63c7e8f7f44e31e65b1204f76f775dee7f8b72d4d4956522'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('/kaggle/input/cleaned-dataset/cleaned_dataset.csv')

In [ ]:
# Split into training and validation sets (80% training, 20% validation)
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['target'])

# Train on each chunk sequentially
chunk_size = len(train_df) // 5
chunks = [train_df[i * chunk_size : (i + 1) * chunk_size] for i in range(5)]

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [ ]:
# import spacy

# nlp = spacy.load('en_core_web_sm')

# # Lemmatization using spaCy
# train_df['cleaned_text'] = train_df['cleaned_text'].apply(lambda x: [token.lemma_ for token in nlp(' '.join(x))])

# stop_words = set(stopwords.words('english'))
# train_df['cleaned_text'] = train_df['cleaned_text'].apply(lambda x: [token for token in x if token.lower() not in stop_words])

max_words = 10000
tokenizer_1 = Tokenizer(num_words=max_words)
tokenizer_1.fit_on_texts(train_df['cleaned_text'])

max_words_2 = 25000
tokenizer_2 = Tokenizer(num_words=max_words_2)
tokenizer_2.fit_on_texts(train_df['cleaned_text'])

X_train_seq_1 = tokenizer_1.texts_to_sequences(train_df['cleaned_text'])
X_val_seq_1 = tokenizer_1.texts_to_sequences(val_df['cleaned_text'])

X_train_seq_2 = tokenizer_2.texts_to_sequences(train_df['cleaned_text'])
X_val_seq_2 = tokenizer_2.texts_to_sequences(val_df['cleaned_text'])

max_sequence_length_1 = max([len(seq) for seq in X_train_seq_1])
X_train_pad_1 = pad_sequences(X_train_seq_1, maxlen=max_sequence_length_1)
X_val_pad_1 = pad_sequences(X_val_seq_1, maxlen=max_sequence_length_1)

max_sequence_length_2 = max([len(seq) for seq in X_train_seq_2])
X_train_pad_2 = pad_sequences(X_train_seq_2, maxlen=max_sequence_length_2)
X_val_pad_2 = pad_sequences(X_val_seq_2, maxlen=max_sequence_length_2)

In [ ]:
train_df.head()

,cleaned_text,target
551135,sore hair toe nail cheerleading even walk down...,0
755105,go mile per hour rest we enjoy spring,4
105947,know nick look beautiful wait long,0
598000,overslept bahhh way airport,0
711830,job torture,0


In [ ]:
def create_model(max_sequence_length, embedding_dim, max_words, lstm_units, num_layers=1):
    model = Sequential()
    model.add(Embedding(max_words, embedding_dim, input_length=max_sequence_length))
    for _ in range(num_layers - 1):
        model.add(LSTM(lstm_units, return_sequences=True))
    model.add(LSTM(lstm_units, return_sequences=False))
    model.add(Dense(5, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
def train_model(model, X_train_pad, y_train, X_val_pad, y_val, batch_size):
    model.fit(X_train_pad, y_train, batch_size=batch_size, epochs=10, validation_data=(X_val_pad, y_val))

In [ ]:
# Set hyperparameters for model set 1
set1_params = {
    'batch_size': 4,
    'max_sequence_length': max_sequence_length_1,
    'embedding_dim': 10,
    'max_words': 10000,
    'lstm_units': 8
}

# Build model set 1
model_set1 = create_model(set1_params['max_sequence_length'], set1_params['embedding_dim'], set1_params['max_words'], set1_params['lstm_units'], num_layers=1)

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
# Train the model
train_model(model_set1, X_train_pad_1, train_df['target'], X_val_pad_1, val_df['target'], batch_size=set1_params['batch_size'])

Epoch 1/10
292203/292203 ━━━━━━━━━━━━━━━━━━━━ 1441s 5ms/step - accuracy: 0.7459 - loss: 0.5108 - val_accuracy: 0.7680 - val_loss: 0.4794
Epoch 2/10
292203/292203 ━━━━━━━━━━━━━━━━━━━━ 1439s 5ms/step - accuracy: 0.7724 - loss: 0.4734 - val_accuracy: 0.7698 - val_loss: 0.4768
Epoch 3/10
292203/292203 ━━━━━━━━━━━━━━━━━━━━ 1437s 5ms/step - accuracy: 0.7774 - loss: 0.4660 - val_accuracy: 0.7698 - val_loss: 0.4774
Epoch 4/10
292203/292203 ━━━━━━━━━━━━━━━━━━━━ 1437s 5ms/step - accuracy: 0.7781 - loss: 0.4644 - val_accuracy: 0.7690 - val_loss: 0.4780
Epoch 5/10
292203/292203 ━━━━━━━━━━━━━━━━━━━━ 1434s 5ms/step - accuracy: 0.7793 - loss: 0.4638 - val_accuracy: 0.7696 - val_loss: 0.4790
Epoch 6/10
292203/292203 ━━━━━━━━━━━━━━━━━━━━ 1437s 5ms/step - accuracy: 0.7802 - loss: 0.4621 - val_accuracy: 0.7683 - val_loss: 0.4841
Epoch 7/10
292203/292203 ━━━━━━━━━━━━━━━━━━━━ 1444s 5ms/step - accuracy: 0.7804 - loss: 0.4622 - val_accuracy: 0.7684 - val_loss: 0.4801
Epoch 8/10
292203/292203 ━━━━━━━━━━━━━━━━

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Evaluate on validation data
y_val_pred_set1 = model_set1.predict(X_val_pad_1)

# Convert predicted probabilities to class labels
y_val_pred_set1_classes = y_val_pred_set1.argmax(axis=1)

print("Set 1 Classification Report:")
print(classification_report(val_df['target'], y_val_pred_set1_classes))

print("Set 1 Confusion Matrix:")
print(confusion_matrix(val_df['target'], y_val_pred_set1_classes))

9132/9132 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step
Set 1 Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.75      0.77    147804
           4       0.75      0.78      0.77    144399

    accuracy                           0.77    292203
   macro avg       0.77      0.77      0.77    292203
weighted avg       0.77      0.77      0.77    292203

Set 1 Confusion Matrix:
[[110919  36885]
 [ 31125 113274]]


In [ ]:
# Set hyperparameters for model set 2
set2_params = {
    'batch_size': 8,
    'max_sequence_length': max_sequence_length_2,
    'embedding_dim': 30,
    'max_words': 25000,
    'lstm_units': 16
}

# Build model set 2 (two LSTM layers)
model_set2 = create_model(set2_params['max_sequence_length'], set2_params['embedding_dim'], set2_params['max_words'], set2_params['lstm_units'], num_layers=2)

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
# Train the model
train_model(model_set2, X_train_pad_2, train_df['target'], X_val_pad_2, val_df['target'], batch_size=set2_params['batch_size'])

Epoch 1/10
146102/146102 ━━━━━━━━━━━━━━━━━━━━ 1010s 7ms/step - accuracy: 0.7490 - loss: 0.5063 - val_accuracy: 0.7731 - val_loss: 0.4738
Epoch 2/10
146102/146102 ━━━━━━━━━━━━━━━━━━━━ 996s 7ms/step - accuracy: 0.7828 - loss: 0.4574 - val_accuracy: 0.7744 - val_loss: 0.4705
Epoch 3/10
146102/146102 ━━━━━━━━━━━━━━━━━━━━ 999s 7ms/step - accuracy: 0.7901 - loss: 0.4463 - val_accuracy: 0.7745 - val_loss: 0.4710
Epoch 4/10
146102/146102 ━━━━━━━━━━━━━━━━━━━━ 998s 7ms/step - accuracy: 0.7963 - loss: 0.4369 - val_accuracy: 0.7741 - val_loss: 0.4750
Epoch 5/10
146102/146102 ━━━━━━━━━━━━━━━━━━━━ 988s 7ms/step - accuracy: 0.8011 - loss: 0.4295 - val_accuracy: 0.7716 - val_loss: 0.4803
Epoch 6/10
146102/146102 ━━━━━━━━━━━━━━━━━━━━ 990s 7ms/step - accuracy: 0.8049 - loss: 0.4232 - val_accuracy: 0.7710 - val_loss: 0.4773
Epoch 7/10
146102/146102 ━━━━━━━━━━━━━━━━━━━━ 1014s 7ms/step - accuracy: 0.8083 - loss: 0.4188 - val_accuracy: 0.7691 - val_loss: 0.4843
Epoch 8/10
146102/146102 ━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
# Evaluate on validation data for model_set2
y_val_pred_set2 = model_set2.predict(X_val_pad_2)

# Convert predicted probabilities to class labels
y_val_pred_set2_classes = y_val_pred_set2.argmax(axis=1)

print("Set 2 Classification Report:")
print(classification_report(val_df['target'], y_val_pred_set2_classes))

print("Set 2 Confusion Matrix:")
print(confusion_matrix(val_df['target'], y_val_pred_set2_classes))

9132/9132 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step
Set 2 Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.78      0.77    147804
           4       0.77      0.76      0.76    144399

    accuracy                           0.77    292203
   macro avg       0.77      0.77      0.77    292203
weighted avg       0.77      0.77      0.77    292203

Set 2 Confusion Matrix:
[[114660  33144]
 [ 35341 109058]]


In [ ]:
!apt-get install texlive texlive-xetex texlive-latex-extra pandoc
!pip install pypandoc

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  dvisvgm fonts-droid-fallback fonts-lato fonts-lmodern fonts-noto-mono fonts-texgyre
  fonts-urw-base35 libapache-pom-java libcmark-gfm-extensions0.29.0.gfm.3 libcmark-gfm0.29.0.gfm.3
  libcommons-logging-java libcommons-parent-java libfontbox-java libfontenc1 libgs9 libgs9-common
  libidn12 libijs-0.35 libjbig2dec0 libkpathsea6 libpdfbox-java libptexenc1 libruby3.0 libsynctex2
  libteckit0 libtexlua53 libtexluajit2 libwoff1 libzzip-0-13 lmodern pandoc-data poppler-data
  preview-latex-style rake ruby ruby-net-telnet ruby-rubygems ruby-webrick ruby-xmlrpc ruby3.0
  rubygems-integration t1utils teckit tex-common tex-gyre texlive-base texlive-binaries
  texlive-fonts-recommended texlive-latex-base texlive-latex-recommended texlive-pictures
  texlive-plain-generic tipa xfonts-encodings xfonts-utils
Suggested packages:
  fonts-noto fonts-fre

In [ ]:
!jupyter nbconvert --to PDF "/content/drive/My Drive/Colab Notebooks/nlp-2"

[NbConvertApp] Converting notebook /content/drive/My Drive/Colab Notebooks/nlp-2 to PDF
[NbConvertApp] Writing 79066 bytes to notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: ['xelatex', 'notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: ['bibtex', 'notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 64190 bytes to /content/drive/My Drive/Colab Notebooks/nlp-.pdf


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
